# Extract profiles (Aschwanden, Truffer, and Fahnestock, 2016)

In [ ]:
import cartopy.crs as ccrs

import geopandas as gp
from joblib import Parallel, delayed
import numpy as np
import pandas as pd
from pathlib import Path
import pylab as plt
from matplotlib import colors, cm
from matplotlib.colors import LightSource
import matplotlib.ticker as mticker
from tqdm.auto import tqdm
import xarray as xr

from pypism import profiles
from pypism.profiles import process_profile
from pypism.utils import preprocess_nc
from pypism.hillshade import hillshade
from pypism.utils import qgis2cmap, tqdm_joblib, blend_multiply

## Resolution along profiles

In [ ]:
profile_resolution = 200 # m

## Load profiles and segmentize

In [ ]:
profiles_path = Path("../data/greenland-flux-gates.gpkg")
profiles_gp = gp.read_file(profiles_path).rename(columns={"id": "profile_id"})
geom = profiles_gp.segmentize(profile_resolution)
profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)

## Load observed velocities

In [ ]:
obs_file = Path("/Users/andy/Google Drive/My Drive/data/ITS_LIVE/GRE_G0240_0000.nc")
obs_ds = xr.open_dataset(obs_file)

## Load Ensemble Experiments

In [ ]:
pism_files = list(Path("/Users/andy/Google Drive/My Drive/Projects/gris-calib/data").glob("velsurf_mag_gris*.nc"))
sim_ds = xr.open_mfdataset(pism_files, 
                  preprocess=preprocess_nc,
                  concat_dim="exp_id",
                  combine="nested",
                  parallel=True)

## Extract all profiles

In [ ]:
xr.apply_ufunc?

## Plot profiles

In [ ]:
run ../profiles/compute_profiles.py --thickness_url "~/Google Drive/My Drive/data/MCdataset/BedMachineGreenland-v5.nc" --n_jobs 4 --alpha 0.05 --result_dir ../profiles/2024_04_test --velocity_url GRE_G0240_0000.nc --profiles_url ../data/greenland-flux-gates.gpkg "~/Google Drive/My Drive/Projects/gris-calib/2022_09_*/state/gris_g1800m_v4_id_0_0_50.nc"

In [ ]:
exp_ds

In [ ]:
flux_x_da.name = "mass_flux_x"

In [ ]:
from typing import Union
import numpy as np
import cartopy.crs as ccrs
import geopandas as gp
import numpy as np
import pylab as plt
import seaborn as sns
import xarray as xr
from matplotlib import cm, colors
from matplotlib.colors import LightSource

from glacier_flow_tools.utils import blend_multiply, figure_extent, get_dataarray_extent


In [ ]:
os = obs_sims_profiles[0]

In [ ]:
flux_x_da

In [ ]:
from pathlib import Path
from typing import Dict, List, Union

import cartopy.crs as ccrs
import geopandas as gp
import numpy as np
import pylab as plt
import seaborn as sns
import xarray as xr
from matplotlib import cm, colors
from matplotlib.colors import LightSource
from glacier_flow_tools.utils import blend_multiply, figure_extent, get_dataarray_extent


In [ ]:
def plot_glacier(
    profile_series: gp.GeoSeries,
    surface: xr.DataArray,
    overlay: xr.DataArray,
    result_dir: Union[str, Path],
    cmap="viridis",
    vmin: float = 10,
    vmax: float = 1500,
    ticks: Union[List[float], np.ndarray] = [10, 100, 250, 500, 750, 1500],
):
    """
    Plot a surface over a hillshade, add profile and correlation coefficient.

    This function plots a surface over a hillshade, adds a profile and correlation coefficient.
    The plot is saved as a PDF file in the specified result directory.

    Parameters
    ----------
    profile_series : gp.GeoSeries
        The profile to be plotted.
    surface : xr.DataArray
        The surface to be plotted over the hillshade.
    overlay : xr.DataArray
        The overlay to be added to the plot.
    result_dir : Union[str, Path]
        The directory where the result PDF file will be saved.
    cmap : str, optional
        The colormap to be used for the plot, by default "viridis".
    vmin : float, optional
        The minimum value for the colormap, by default 10.
    vmax : float, optional
        The maximum value for the colormap, by default 1500.
    ticks : Union[List[float], np.ndarray], optional
        The ticks to be used for the colorbar, by default [10, 100, 250, 500, 750, 1500].
    """

    geom = getattr(profile_series, "geometry")
    geom_centroid = geom.centroid
    profile_centroid = gp.GeoDataFrame([profile_series], geometry=[geom_centroid])
    profile = gp.GeoDataFrame([profile_series], geometry=[geom])
    glacier_name = getattr(profile, "profile_name").values[0]
    exp_id = getattr(profile, "exp_id").values[0]
    x_c = round(profile_centroid.geometry.x.values[0])
    y_c = round(profile_centroid.geometry.y.values[0])
    extent_slice = figure_extent(x_c, y_c)
    cartopy_crs = ccrs.NorthPolarStereo(central_longitude=-45, true_scale_latitude=70, globe=None)
    # Shade from the northwest, with the sun 45 degrees from horizontal
    light_source = LightSource(azdeg=315, altdeg=45)
    glacier_overlay = overlay.sel(extent_slice)
    glacier_surface = surface.interp_like(glacier_overlay)

    extent = get_dataarray_extent(glacier_overlay)
    norm = colors.Normalize(vmin=vmin, vmax=vmax)
    mapper = cm.ScalarMappable(norm=norm, cmap=cmap)

    v = mapper.to_rgba(glacier_overlay.to_numpy())
    z = glacier_surface.to_numpy()
    fig = plt.figure(figsize=(6.2, 6.2))
    ax = fig.add_subplot(111, projection=cartopy_crs)
    rgb = light_source.shade_rgb(v, elevation=z, vert_exag=0.01, blend_mode=blend_multiply)
    # Use a proxy artist for the colorbar...
    im = ax.imshow(v, cmap=cmap, vmin=vmin, vmax=vmax)
    im.remove()
    corr = ax.imshow(
        v,
        vmin=0,
        vmax=1,
        cmap="RdYlGn",
    )
    corr.remove()
    ax.imshow(rgb, extent=extent, origin="upper", transform=cartopy_crs)
    profile.plot(ax=ax, color="k", lw=1)
    profile_centroid.plot(
        column="pearson_r", vmin=0, vmax=1, cmap="RdYlGn", markersize=50, legend=False, missing_kwds={}, ax=ax
    )
    ax.annotate(f"{glacier_name}", (x_c, y_c), (10, 10), xycoords="data", textcoords="offset points")
    ax.gridlines(
        draw_labels={"top": "x", "left": "y"},
        dms=True,
        xlocs=np.arange(-50, 0, 1),
        ylocs=np.arange(50, 88, 1),
        x_inline=False,
        y_inline=False,
        rotate_labels=20,
        ls="dotted",
        color="k",
    )

    ax.set_extent(extent, crs=cartopy_crs)
    fig.colorbar(im, ax=ax, shrink=0.5, pad=0.025, label=overlay.units, extend="max", ticks=ticks)
    fig.colorbar(
        corr, ax=ax, shrink=0.5, pad=0.025, label="Pearson $r$ (1)", orientation="horizontal", location="bottom"
    )
    fig.savefig(result_dir / Path(f"{glacier_name}_{exp_id}_speed.pdf"))
    plt.close()
    del fig


In [ ]:
from shapely import LineString

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gp
from typing import Dict

In [ ]:
profiles_dict = {'profile_id': {0: 0, 1: 1},
 'profile_name': {0: 'Horizontal Gletscher', 1: 'Vertical'},
 'geometry': {0: LineString([[-10, 0], [10, 0]]),
  1: LineString([[0, -10],[0, 10]])}}
profiles_gp = gp.GeoDataFrame.from_dict(profiles_dict)

In [ ]:
from shapely import get_coordinates


In [ ]:
        nx = 201
        ny = 201
        x_min = -10
        x_max = 10
        y_min = -10
        y_max = 10
        x = np.linspace(x_min, x_max, nx)
        y = np.linspace(y_min, y_max, ny)
        X, Y = np.meshgrid(x, y)
    
        rng = np.random.default_rng(seed=42)
        # Directional vectors
        vx = np.zeros_like(X)
        vy = (y_max - Y**2) + rng.random(size=Y.shape)

In [ ]:
rng.random(size=Y.size).shape

In [ ]:
from shapely import get_coordinates

In [ ]:
import xarray as xr

In [ ]:

    nx = 201
    ny = 201
    x_min = -10
    x_max = 10
    y_min = -10
    y_max = 10
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    rng = np.random.default_rng(seed=42)
    # Directional vectors
    vx = (y_max**2 - Y**2) + rng.random(size=Y.shape)
    vy = np.zeros_like(X)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    quadratic_flow = ds

In [ ]:
from numpy.testing import assert_array_almost_equal

In [ ]:
    from glacier_flow_tools import profiles
    profiles_dict = {
        "profile_id": {0: 0, 1: 1},
        "profile_name": {0: "Horizontal Gletscher", 1: "Vertical"},
        "geometry": {0: LineString([[-10, 0], [10, 0]]), 1: LineString([[0, -10], [0, 10]])},
    }
    profiles_gp = gp.GeoDataFrame.from_dict(profiles_dict)
    geom = profiles_gp.segmentize(1.0)
    profiles_gp = gp.GeoDataFrame(profiles_gp, geometry=geom)
    profiles_gp = profiles_gp[["profile_id", "profile_name", "geometry"]]

    profile = profiles_gp.loc[[0]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]

    kwargs: Dict = {}
    x_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)

    profile = profiles_gp.loc[[1]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]
    x_vy_true = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])
    assert_array_almost_equal(x_profile.vy, x_vy_true)


In [ ]:
x_profile.vy

In [ ]:
kwargs.update({"compute_profile_normal": True})

In [ ]:
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
    profile = profiles_gp.loc[[1]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]
    x_vy_true = np.array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])
    assert_array_almost_equal(x_profile.vy, x_vx_true)

    kwargs: Dict = {}
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)
array([  0.90858069,  19.51916723,  36.95104485,  51.28749629,
        64.28259608,  75.47610046,  84.02253044,  91.83760948,
        96.92277666,  99.93217176, 100.29915991,  99.22948628,
        96.05096098,  91.20227129,  84.15173917,  75.92502223,
        64.51036844,  51.41748735,  36.1520003 ,  19.53233673,
         0.23981163])

In [ ]:
    y_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
normal_var = "v_normal"
normal_component_vars: dict = {"x": "vx", "y": "vy"}

In [ ]:
%load_ext rich

In [ ]:
y_profile.vx.to_numpy()

In [ ]:
    profile = profiles_gp.loc[[0]]
    geom = getattr(profile, "geometry")
    x_p, y_p = get_coordinates(geom).T
    profile_name = getattr(profile, "profile_name").values[0]
    profile_id = getattr(profile, "profile_id").values[0]

    kwargs: Dict = {}
    x_profile = quadratic_flow.profiles.extract_profile(x_p, y_p, profile_name=profile_name, profile_id=profile_id, **kwargs)


In [ ]:
    nx = 201
    ny = 201
    x_min = -10
    x_max = 10
    y_min = -10
    y_max = 10
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    rng = np.random.default_rng(seed=42)
    # Directional vectors
    vy = np.zeros_like(Y)
    vx = (y_max**2 - Y**2) + rng.random(size=Y.shape)


In [ ]:
plt.imshow(vx)
plt.colorbar()

In [ ]:
import pylab as plt

In [ ]:
plt.imshow(-(-Y**2))
plt.colorbar()

In [ ]:
plt.plot(y, y_max -y**2)

In [ ]:
kwargs

In [ ]:
cf_xarray.units.units.kelvin

In [ ]:
import xarray as xr
import cf_xarray.units
import pint_xarray
ds = xr.open_dataset("../pism_test_pint_xarray.nc")
for v in ds.data_vars:
    da = ds[v]
    da.pint.quantify()

In [ ]:
ds = xr.open_mfdataset("../pism_test_pint_xarray*.nc").drop_vars(["shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav"]).chunk("auto").pint.quantify()

In [ ]:
from glacier_flow_tools.utils import preprocess_nc
from functools import partial
ds = xr.open_mfdataset("../gris_g1800m_v4_id_*_0_50.nc",
                       preprocess=partial(preprocess_nc, drop_dims=["z", "zb"], drop_vars=["timestamp", "shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav"]),
        concat_dim="exp_id",
        combine="nested",
        chunks="auto",
        engine="h5netcdf",
        parallel=True,
        decode_times=False,
    )


In [ ]:
ds1 = xr.open_dataset("../gris_g1800m_v4_id_bar_0_50.nc").chunk("auto")
ds2 = xr.open_dataset("../gris_g1800m_v4_id_foo_0_50.nc").chunk("auto")

In [ ]:
ds1 = xr.open_dataset("~/Google Drive/My Drive/Projects/gris-calib/2022_09_thermo/state/gris_g1800m_v4_id_0_0_50.nc").chunk("auto")
ds2 = xr.open_dataset("~/Google Drive/My Drive/Projects/gris-calib/2022_09_thermo/state/gris_g1800m_v4_id_0_0_50.nc").chunk("auto")

In [ ]:
ds1_p = preprocess_nc(ds1, drop_dims=["z", "zb"], drop_vars=["timestamp", "shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav"])
ds2_p = preprocess_nc(ds2, drop_dims=["z", "zb"], drop_vars=["timestamp", "shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav"])

In [ ]:
merged  = xr.concat([ds1_p, ds2_p], dim="exp_id")

In [ ]:
merged.pint.quantify().thk

In [ ]:
ds = xr.open_mfdataset("~/Google Drive/My Drive/Projects/gris-calib/2022_09_*/state/gris_g1800m_v4_id_0_0_50.nc",
                       preprocess=partial(preprocess_nc, drop_dims=["z", "zb"], drop_vars=["timestamp", "shelfbtemp", "effective_ice_surface_temp", "ice_surface_temp", "hardav"]),
        concat_dim="exp_id",
        combine="nested",
        chunks="auto",
        engine="h5netcdf",
        parallel=True,
        decode_times=False,
    )

def preprocess_nc(
    ds: xr.Dataset,
    regexp: str = "id_(.+?)_",
    dim: str = "exp_id",
    drop_vars: Union[str, Iterable[Hashable], Callable[[xr.Dataset], Union[str, Iterable[Hashable]]]] = ["nv4"],
    drop_dims: List[str] = ["nv4"],
) -> xr.Dataset:
    """
    Add experiment 'exp_id' to the dataset and drop specified variables and dimensions.

    This function adds an experiment id ('exp_id') to the dataset, extracted from the source encoding
    using the provided regular expression. It then drops the specified variables and dimensions from the dataset.

    Parameters
    ----------
    ds : xr.Dataset
        The dataset to be preprocessed.
    regexp : str, optional
        The regular expression used to extract the experiment id from the source encoding, by default "id_(.+?)_".
    dim : str, optional
        The name of the dimension to be added to the dataset, by default "exp_id".
    drop_vars : Union[List[str], None], optional
        The variables to be dropped from the dataset, by default None.
    drop_dims : List[str], optional
        The dimensions to be dropped from the dataset, by default ["nv4"].

    Returns
    -------
    xr.Dataset
        The preprocessed dataset.
    """

    m_id_re = re.search(regexp, ds.encoding["source"])
    ds.expand_dims(dim)
    assert m_id_re is not None
    m_id: Union[str, int]
    try:
        m_id = int(m_id_re.group(1))
    except:
        m_id = str(m_id_re.group(1))
    ds[dim] = m_id
    return ds.drop_vars(drop_vars, errors="ignore").drop_dims(drop_dims, errors="ignore")



In [ ]:
ds.pint.quantify()

In [ ]:
merged.pint.dequantify()

In [ ]:
ds.pint.dequantify()

In [ ]:
merged.thk

In [ ]:
ds.thk.pint.quantify()

In [ ]:
import pint

In [ ]:
ureg = pint.UnitRegistry()

In [ ]:
ureg.parse_units("m")

In [ ]:
ureg.parse_units_as_container(ds.thk.attrs["units"])

In [ ]:
ureg.m

In [ ]:
ds.thk.attrs

In [ ]:
merged.thk.attrs

In [ ]:
in_ds = xr.open_dataset("../bar.nc")

In [ ]:
in_ds.to_dict()

In [ ]:
import cf_xarray.units
import pint_xarray

import cftime
import xarray as xr
import re

def preprocess_nc(
    ds: xr.Dataset,
    regexp: str = "id_(.+?)_",
    dim: str = "exp_id",
) -> xr.Dataset:
    """
    Add experiment 'exp_id' to the dataset.

    This function adds an experiment id ('exp_id') to the dataset, extracted from the source encoding
    using the provided regular expression.

    Parameters
    ----------
    ds : xr.Dataset
        The dataset to be preprocessed.
    regexp : str, optional
        The regular expression used to extract the experiment id from the source encoding, by default "id_(.+?)_".
    dim : str, optional
        The name of the dimension to be added to the dataset, by default "exp_id".

    Returns
    -------
    xr.Dataset
        The preprocessed dataset.
    """

    m_id_re = re.search(regexp, ds.encoding["source"])
    ds.expand_dims(dim)
    assert m_id_re is not None
    m_id: Union[str, int]
    try:
        m_id = int(m_id_re.group(1))
    except:
        m_id = str(m_id_re.group(1))
    ds[dim] = m_id
    return ds


d = {'coords': {'lat': {'dims': ('y', 'x'),
   'attrs': {'units': 'degree_north',
    'valid_range': [-90.0, 90.0],
    'long_name': 'latitude',
    'standard_name': 'latitude'},
   'data': [[0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0]]},
  'lon': {'dims': ('y', 'x'),
   'attrs': {'units': 'degree_east',
    'valid_range': [-180.0, 180.0],
    'long_name': 'longitude',
    'standard_name': 'longitude'},
   'data': [[0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0],
    [0.0, 0.0, 0.0, 0.0, 0.0]]},
  'time': {'dims': ('time',),
   'attrs': {'axis': 'T', 'long_name': 'time'},
   'data': [cftime.DatetimeNoLeap(1001, 1, 1, 0, 0, 0, 0, has_year_zero=True)]},
  'x': {'dims': ('x',),
   'attrs': {'units': 'm',
    'axis': 'X',
    'long_name': 'X-coordinate in Cartesian system',
    'standard_name': 'projection_x_coordinate',
    'spacing_meters': 375000.0},
   'data': [-750000.0, -375000.0, 0.0, 375000.0, 750000.0]},
  'y': {'dims': ('y',),
   'attrs': {'units': 'm',
    'axis': 'Y',
    'long_name': 'Y-coordinate in Cartesian system',
    'standard_name': 'projection_y_coordinate',
    'spacing_meters': 375000.0},
   'data': [-750000.0, -375000.0, 0.0, 375000.0, 750000.0]}},
 'attrs': {'Conventions': 'CF-1.6'},
 'dims': {'y': 5, 'x': 5, 'time': 1},
 'data_vars': {'thk': {'dims': ('time', 'y', 'x'),
   'attrs': {'units': 'm',
    'valid_min': 0.0,
    'long_name': 'land ice thickness',
    'standard_name': 'land_ice_thickness'},
   'data': [[[0.0, 0.0, 0.0, 0.0, 0.0],
     [0.0, 0.0, 499.6684394216143, 0.0, 0.0],
     [0.0, 499.6684394216143, 499.66844085642595, 499.6684394216143, 0.0],
     [0.0, 0.0, 499.6684394216143, 0.0, 0.0],
     [0.0, 0.0, 0.0, 0.0, 0.0]]]}}}

ds = xr.Dataset.from_dict(d)

file1 = "test_id_foo_0_1000.nc"
file2 = "test_id_bar_0_1000.nc"

ds.to_netcdf(file1)
ds.to_netcdf(file2)

ds1 = preprocess_nc(xr.open_dataset(file1)).chunk("auto")
ds2 = preprocess_nc(xr.open_dataset(file2)).chunk("auto")
single_ds = xr.concat([ds1, ds2], dim="exp_id")
single_ds.pint.quantify()

mf_ds = xr.open_mfdataset("test_id_*.nc",
                          preprocess=preprocess_nc,
                          concat_dim="exp_id",
                          combine="nested",
                          chunks="auto",
                          parallel=False,
                          decode_times=False)

mf_ds.pint.quantify()

In [ ]:
print("xarray.__version__ = ", xr.__version__)
print("pint_xarray.__version__ = ", pint_xarray.__version__)
print("cf_xarray.__version__ = ", cf_xarray.__version__)


In [ ]:
cf_xarray.units.units.Celsius